In [1]:
#All the imports & pyplot inline setting
import numpy as np
import pandas as pd

In [2]:
#Input parameters
n_file='physics_collaboration_net.txt'
n_file_net = 'physics_collaboration_net.net'
n_power = 2
n_inflation = 1.27
#n_file='attweb_net.txt'
#n_file_net='attweb_net.net'
#n_power = 2
#n_inflation = 1.35
#n_file='yeast_undirected_metabolic.txt'
#n_file_net='yeast_undirected_metabolic.net'
#n_power = 2
#n_inflation = 1.205

In [4]:
#Process net file to get vertice order in dictionary res
with open(n_file_net) as f:
    content = f.readlines()
out = []
for i,data in enumerate(content):
    if data.startswith('*Vertices'):
        continue
    if data.startswith('*Edges'):
        break
    out.append(data.replace('\"','').split())
df = pd.DataFrame(out)
df = df[[0,1]]
a = df.to_dict()[1]
res = dict((v,k) for k,v in a.iteritems())

In [5]:
#Process input file and created normalised adjacency list
n_data = df.shape[0]
df = pd.read_csv(n_file,header=None,delim_whitespace=True,dtype={0: object, 1: object})
df[0] = df[0].astype('object')
df[1] = df[1].astype('object')
adjacency_matrix = np.zeros((n_data,n_data))
np.fill_diagonal(adjacency_matrix,1)
for i in range(df.shape[0]):
    j = res[df.loc[i][0]]
    k = res[df.loc[i][1]]
    adjacency_matrix[j][k] = 1
    adjacency_matrix[k][j] = 1
old = adjacency_matrix
new = np.divide(adjacency_matrix,np.sum(adjacency_matrix, axis=0))

In [6]:
#Alternate expansion and inflation
i=0
while(not (old==new).all() and i<=1000):
    old = new
    new = np.linalg.matrix_power(new, n_power)
    new = np.power(new, n_inflation)
    new = np.divide(new,np.sum(new, axis=0))
    #new = new.round(3)
    i = i + 1

In [7]:
#Check no of iterations required to merge
i

55

In [8]:
#Generate Clusters
clusters = []
for i in range(n_data):
    if((new[i]>0).any()):
        clusters.append(np.nonzero(new[i])[0])
clusters

[array([ 0,  1,  2,  3, 80, 84, 85, 86], dtype=int64),
 array([  4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  72,  73,
         74,  75,  76, 110, 111, 112, 113, 114, 124, 130, 140, 141], dtype=int64),
 array([ 22,  25,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
         38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
         51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
         64,  65,  66,  81,  82,  83,  87,  88,  89,  90,  91,  92,  93,
         94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 107, 115,
        116, 125, 134, 135, 139], dtype=int64),
 array([ 77,  78,  79, 117, 118, 119, 120, 121], dtype=int64),
 array([ 15,  16,  17,  18,  19,  20,  21,  23,  24,  26,  67,  68,  69,
         70,  71, 105, 106, 108, 109, 122, 123, 126, 127, 128, 129, 131,
        132, 133, 136, 137, 138], dtype=int64)]

In [9]:
#Create labels
out_label = np.zeros((n_data,1), dtype=np.int)
for i,data in enumerate(clusters):
    for fine_data in data:
        out_label[data] = i+1
out_label

array([[1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [3],
       [5],
       [5],
       [3],
       [5],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [5],
       [5],
       [5],
       [5],
       [5],
       [2],
       [2],
       [2],
       [2],
       [2],
       [4],
       [4],
       [4],
       [1],
       [3],
       [3],
    

In [10]:
#Save clu file
np.savetxt('physics_2_1.27.clu',out_label,fmt='%d',header='*vertices '+str(n_data),comments='')
#np.savetxt('att_2_1.35.clu',out_label,fmt='%d',header='*vertices '+str(n_data),comments='')
#np.savetxt('yeast_2_1.205.clu',out_label,fmt='%d',header='*vertices '+str(n_data),comments='')